In [1]:
from qdrant_client import QdrantClient, models

In [2]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [10]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-rag"

In [6]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [8]:
import random,json

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece, indent=2))

{
  "text": "Origin of Solution (Mage Slack-Channel): https://mageai.slack.com/archives/C03HTTWFEKE/p1706543947795599\nProblem: This error can often be seen after solving the error mentioned in 2.2.4. The error can be found in Mage version 0.9.61 and is a side-effect of the update of the code for data-loader blocks.\nNote: Mage 0.9.62 has been released, as of Feb 5 2024. Please recheck. Solution below may be obsolete\nSolution: Using a \u201cfixed\u201d version of the docker container\nPull updated docker image from docker-hub\nmageai/mageaidocker pull:alpha\nUpdate docker-compose.yaml\nversion: '3'\nservices:\nmagic:\nimage: mageai/mageai:alpha  <--- instead of \u201clatest\u201d-tag\ndocker-compose up\nThe original Error is still present, but the SQL-query will return the desired result:\n--------------------------------------------------------------------------------------",
  "section": "Module 2: Workflow Orchestration",
  "question": "Mage - 2.2.4 IndexError: list index out of ra

In [11]:
result = search(course_piece['question'])

In [12]:
result

QueryResponse(points=[ScoredPoint(id=878, version=0, score=0.8007077, payload={'text': 'Running “python hpo.py --data_path=./your-path --max_evals=50” for the homework leads to the following error: TypeError: unsupported operand type(s) for -: \'str\' and \'int\'\nFull Traceback:\nFile "~/repos/mlops/02-experiment-tracking/homework/hpo.py", line 73, in <module>\nrun(args.data_path, args.max_evals)\nFile "~/repos/mlops/02-experiment-tracking/homework/hpo.py", line 47, in run\nfmin(\nFile "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/fmin.py", line 540, in fmin\nreturn trials.fmin(\nFile "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/base.py", line 671, in fmin\nreturn fmin(\nFile "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/fmin.py", line 586, in fmin\nrval.exhaust()\nFile "~/Library/Caches/pypoetry/virtualenvs/mlfl

In [13]:
print(f"Question:\n{course_piece['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_piece['text']))

Question:
Mage - 2.2.4 IndexError: list index out of range

Top Retrieved Answer:
Running “python hpo.py --data_path=./your-path --max_evals=50” for the homework leads to the following error: TypeError: unsupported operand type(s) for -: 'str' and 'int'
Full Traceback:
File "~/repos/mlops/02-experiment-tracking/homework/hpo.py", line 73, in <module>
run(args.data_path, args.max_evals)
File "~/repos/mlops/02-experiment-tracking/homework/hpo.py", line 47, in run
fmin(
File "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/fmin.py", line 540, in fmin
return trials.fmin(
File "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/base.py", line 671, in fmin
return fmin(
File "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.9/lib/python3.9/site-packages/hyperopt/fmin.py", line 586, in fmin
rval.exhaust()
File "~/Library/Caches/pypoetry/virtualenvs/mlflow-intro-SyTqwt0D-py3.

In [14]:
print(search("What if I submit homeworks late?").points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [19]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [20]:
print(search_in_course("What if I submit homeworks late?", "mlops-zoomcamp").points[0].payload['text'])

Please choose the closest one to your answer. Also do not post your answer in the course slack channel.
